In [ ]:
%pip install  --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental wikipedia neo4j tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 16.3 MB/s eta 0:00:00
   ━

In [ ]:
from google.colab import userdata
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.text_splitter import TokenTextSplitter
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from neo4j import GraphDatabase
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain_core.pydantic_v1 import BaseModel,Field
from langchain.schema.runnable import RunnableBranch, RunnableLambda, RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
import os



In [ ]:
os.environ["NEO4J_URI"] = "neo4j+s://fd919b29.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "nhn2RV52iGrQdn3MfYbhy80Jp_QrzUc3nuLb9XOB4TI"

In [ ]:
graph = Neo4jGraph(
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD")
)
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
len(raw_documents)

23

In [ ]:
raw_documents[:3]

[Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was i

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter=TokenTextSplitter(chunk_size=512,chunk_overlap=24)
documents=text_splitter.split_documents(raw_documents[:3])

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125", )

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

# New Section

In [ ]:
default_cypher="MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
def showGraph(cypher: str=default_cypher):
  driver=GraphDatabase.driver(
      uri=os.environ["NEO4J_URI"],
      auth=(os.environ["NEO4J_USERNAME"],
            os.environ["NEO4J_PASSWORD"]))
  session=driver.session()
  widget=GraphWidget(graph=session.run(cypher).graph())
  widget.node_label_mapping="id"
  display(widget)
  return widget



In [ ]:
showGraph()

GraphWidget(layout=Layout(height='690px', width='100%'))

GraphWidget(layout=Layout(height='690px', width='100%'))

In [ ]:
from typing import Tuple,List,Optional

In [ ]:

from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings


embedding = OpenAIEmbeddings()

# Specify a new index name
vector_index = Neo4jVector.from_existing_graph(
    embedding,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    index_name="my_new_vector_index"
)

In [ ]:
from langchain_core.pydantic_v1 import BaseModel,Field
class Entities(BaseModel):
  names:List[str]=Field(
      description=" All the person,organisations,or business entities ""appear in the text",

  )



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt=ChatPromptTemplate.from_messages(
    [
        (
        "system",
        "You are extracting organistaion and person entities from the text.",
        ),
        (
            "human",
            "Use the format to extract the infomation from following"
            "input: {question}",
        )



    ]
)


In [ ]:
entity_chain=prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question":"Where was Amelia Earhart born?"}).names

['Amelia Earhart']

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [ ]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR(e:__Entity__)ON EACH [e.id]")

In [ ]:
def generate_full_text_query(input: str)-> str:
  full_text_query=""
  words=[el for el in remove_lucene_chars(input).split()if el]
  if words: # Check if words is not empty
    for word in words[:-1]:
      full_text_query+=f"{word}~2 AND"
    full_text_query+=f"{words[-1]}~2" # Use words[-1] instead of word[-1]
  return full_text_query.strip()

In [ ]:
def structured_retriever(question:str)-> str:
  result=""
  entities=entity_chain.invoke({"question":question})
  for entity in entities.names:
    response=graph.query(
        """CALL db.index.fulltext.queryNodes('entity',$query,{limit:2})
        YIELD node,score
        CALL
          {
          WITH node
          MATCH(node)-[r:!MENTIONS]->(neighbor)
          RETURN node.id+ "-"+type(r)+" ->" +node.id AS OUTPUT
          UNION ALL
          WITH node
           MATCH (node)<-[r:!MENTIONS]-(neighbor)
          RETURN neighbor.id +" -"+ type(r)+"->"+node.id AS OUTPUT
        }
        RETURN OUTPUT LIMIT 50
        """,
        {"query":generate_full_text_query(entity)},




    )

    result+="\n".join([el["output"]for el in response])
  return result

In [ ]:
print(structured_retriever("Who is ElizabethI ?"))


In [ ]:
def retriever(question:str):
  print(f"search query:{question}")
  structured_data=structured_retriever(question)
  unstructured_data=[el.page_content for el in vector_index.similarity_search(question)]
  final_data=f"""STRUCTURED DATA:
  {structured_data}
  Unstructured data:
  {"#Document".join(unstructured_data)}
  """
  return final_data


In [ ]:
_template="""Give the following conversation and a follow up question repharase in its original language.
Chat History:{chat_history}
Follow Up Input:{question}
Standalone question:"""

In [ ]:
CONDENSE_QUESTION_PROMPT=PromptTemplate.from_template(_template)

In [ ]:
def _format_chat_history(chat_history:List[Tuple[str,str]])->List:
  buffer=[]
  for human,ai in chat_history:
    buffer.append(HumanMessage(content=human))
    buffer.append(AIMessage(content=ai))

  return buffer

In [ ]:
_search_query=RunnableBranch(
    (
        RunnableLambda(lambda x:bool(x.get("chat_history"))).with_config(
            run_name="Has ChatHistoryCheck"
        ),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])

        )
        |CONDENSE_QUESTION_PROMPT
        |ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    RunnableLambda(lambda x:x["question"]),
)

In [ ]:
template="""Answer the question based on the context:
{context}
Question:{question}
Use natural language to answer the question
Answer:"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnableParallel

In [ ]:
chain=(
    RunnableParallel(
        {
            "context":_search_query | retriever,
            "question":RunnablePassthrough(),

        }
    )
    | prompt
    |llm
    |StrOutputParser()
)

In [ ]:
chain.invoke({"question":"Which house did Elizabeth I live in?"})

search query:Which house did Elizabeth I live in?


'Elizabeth I lived in the Palace of Whitehall.'

In [ ]:
chain.invoke({"question":"Where was Elizabeth I born?"})

search query:Where was Elizabeth I born?


'Elizabeth I was born in Greenwich, England.'

In [ ]:
chain.invoke({"question":"When did  Elizabeth I crowned as queen?"})


search query:When did  Elizabeth I crowned as queen?


'Elizabeth I was crowned as queen on January 15, 1559.'

In [ ]:
chain.invoke({"question":"When did  Elizabeth I died?"})

search query:When did  Elizabeth I died?


'Elizabeth I died on March 24, 1603.'